# PCML Project-1 ~ Team #60

In [20]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [21]:
from proj1_helpers import *
DATA_TRAIN_PATH = '/home/sarigian/Desktop/project1/Data/train.csv' # TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)

print(tX.shape)

(250000, 30)


## Linear regression using gradient descent

## Linear regression using stochastic gradient descent

## Linear regression using stochastic gradient descent

## Least squares regression using normal equations

## Ridge regression using normal equations

## Logistic regression using gradient descent or SGD

## Regularized logistic regression using gradient descent or SGD

## Generate predictions and save ouput in csv format for submission:

In [11]:
DATA_TEST_PATH = '/home/sarigian/Desktop/project1/Data/test.csv' # TODO: download test data and supply path here 
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [31]:
OUTPUT_PATH = '' # TODO: fill in desired name of output file for submission
y_pred = predict_labels(weights, tX_test)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)